In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, clear_output
import json
import os
import pandas as pd
import numpy as np
import pprint

In [2]:
game_type = widgets.Dropdown(
    options = ['Regular', 'Play_off'],
    description = 'Type of the game :',
    disabled = False
)
def handle_game_type(change):
    new_value = change.new


year_of_the_game = widgets.Dropdown(
    options = [2016, 2017, 2018, 2019, 2020],
    description = 'year of the game',
    disabled = False
)
def handle_year_of_the_game(change):
    new_value = change.new

game_id = widgets.IntSlider(
    min= 1,
    max = 1271,
    description='game number',
    disabled=False
)
def handle_game_id(change):
    new_value = change.new

event = widgets.IntSlider(
    min= 1,
    max = 100,
    description='events',
    disabled=False,
)
def handle_event(change):
    new_value = change.new

In [3]:
inters = widgets.VBox([widgets.HBox([game_type, year_of_the_game]),widgets.HBox([game_id, event])])

In [4]:
game_type.observe(handle_game_type)
year_of_the_game.observe(handle_year_of_the_game)  
game_id.observe(handle_game_id)
event.observe(handle_event)

In [5]:
def mainFunction(game_type, year_of_the_game, game_id, event):
    serial_number = 20000 if game_type == 'Regular' else 30000
    json_file = f'{year_of_the_game}0{int(serial_number) + int(game_id)}'
    gtype = 'R' if game_type == 'Regular' else 'P'
    try:
        with open(f'data/{year_of_the_game}/{gtype}/{json_file}.json') as f:
            game_info = json.load(f)
        f.close()


        home_team = game_info['gameData']["teams"]["home"]["abbreviation"]
        away_team = game_info['gameData']["teams"]["away"]["abbreviation"]
        home_team_goal = game_info['liveData']["linescore"]["teams"]["home"]["goals"]
        away_team_goal = game_info['liveData']["linescore"]["teams"]["away"]["goals"]
        home_team_goal_shots_on_goal = game_info['liveData']["linescore"]["teams"]["home"]["shotsOnGoal"]
        away_team_goal_shots_on_goal = game_info['liveData']["linescore"]["teams"]["away"]["shotsOnGoal"]
        home_team_so = game_info['liveData']["linescore"]["shootoutInfo"]["home"]["scores"]
        away_team_so = game_info['liveData']["linescore"]["shootoutInfo"]["away"]["scores"]
        home_team_attempts = game_info['liveData']["linescore"]["shootoutInfo"]["home"]["attempts"]
        away_team_attempts = game_info['liveData']["linescore"]["shootoutInfo"]["away"]["attempts"]
        event_time = game_info['liveData']['plays']['allPlays'][event]["about"]["periodTime"]
        event_des = game_info['liveData']['plays']['allPlays'][event]["result"]["description"]
        period_time = game_info['liveData']['plays']['allPlays'][event]["about"]["periodTime"]    
        period = game_info['liveData']['plays']['allPlays'][event]["about"]["period"]


        print(f"game_id : {game_info['gamePk']}")
        print(f"total games : {len(os.listdir(f'data/{year_of_the_game}/{gtype}'))}")
        print(f"total events : {len(game_info['liveData']['plays']['allPlays'])}")
        print(f"date time {game_info['gameData']['datetime']['dateTime']}")

        home_data = pd.DataFrame({'Home':[home_team, home_team_goal, home_team_goal_shots_on_goal, home_team_so, home_team_attempts],
                                'Away':[away_team, away_team_goal, away_team_goal_shots_on_goal, away_team_so, away_team_attempts]
                                }, index= ['Teams:','Goals:','Shots on goal:','SO:','Attempts'])

        print(home_data)


        fig = plt.figure(figsize=(8, 4))
        img = mpimg.imread('imgs/nhl_rink.png')
        fig, ax = plt.subplots()
        ax.imshow(img,extent=[-100, 100, -42.5, 42.5])
        plt.xlim((-100, 100))
        plt.ylim((-42.5, 42.5))
        my_y_ticks = np.arange(-42.5, 42.5, 21.25)
        plt.yticks(my_y_ticks)
        plt.xlabel('feet')
        plt.ylabel('feet')


        if period % 2 == 1:
            plt.title(str(event) + '\n' + str(period_time) + '  P-' + str(period) + '\n'+ str(away_team) + 'VS ' + str(home_team))
        else:
            plt.title(str(event) + '\n' + str(period_time) + '  P-' + str(period) + '\n'+ str(home_team) + 'VS ' + str(away_team))

        cors = game_info['liveData']['plays']['allPlays'][event]['coordinates']



        try:
            x, y = cors['x'], cors['y']
            plt.plot(x, y, 'bo')
            plt.annotate(' <'+str(x)+', '+str(y)+'>', xy=(x, y))
        except:
            plt.text(-23, 45, "No ball included !", c='r')

        plt.show()
        pprint.pprint(game_info['liveData']['plays']['allPlays'][event]['result'])
    except:
        print("No Data available !")

In [6]:
final = widgets.interactive_output(mainFunction, {'year_of_the_game': year_of_the_game, 'game_type': game_type, 'game_id' : game_id, 'event' : event})
display(inters, final)

Output()